In [1]:
"""testing the multidim wav2vec model against the reference model"""
import torch
import numpy as np
from lightning.pytorch import seed_everything
import seisbench.data as sbd

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers import Wav2Vec2Config
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining

data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0)



2024-06-17 07:12:11,604 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [2]:
model_output = {}
# for model_name in MODEL_NAMES:
config = Wav2Vec2Config.from_pretrained(
  '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/pretrain/model_config_4xdownsample_sinkhorn.json'
)


# config.sinkhorn_quantization_iters = 1

for sinkhorn_quantization_iter in [0, 1, 3, 5]:
  print('------------')
  print(f"Sinkhorn quantization iter: {sinkhorn_quantization_iter}")
  config.sinkhorn_quantization_iters = sinkhorn_quantization_iter
  model = MultiDimWav2Vec2ForPreTraining(config)

  # compute masked indices
  batch_size, num_channels, raw_sequence_length = input_values.shape
  sequence_length = model._get_feat_extract_output_lengths(
    raw_sequence_length).item()

  seed_everything(0)
  mask_time_indices = _compute_mask_indices(
      shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
  )
  sampled_negative_indices = _sample_negative_indices(
      features_shape=(batch_size, sequence_length),
      num_negatives=model.config.num_negatives,
      mask_time_indices=mask_time_indices,
  )
  mask_time_indices = torch.tensor(
    data=mask_time_indices, device=input_values.device, dtype=torch.long)
  sampled_negative_indices = torch.tensor(
    data=sampled_negative_indices, device=input_values.device,
    dtype=torch.long
  )

  with torch.no_grad():
    outputs = model(input_values, mask_time_indices=mask_time_indices,
                    sampled_negative_indices=sampled_negative_indices)


  print(outputs.codevector_perplexity)
  # model_output[f'{model_name}_{model_type}'] = outputs


------------
Sinkhorn quantization iter: 0


Seed set to 0


codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L tensor([[  2.,   0.,   0.,   4.,   5.,   0.,   0.,   8.,   0.,   0.,   0.,   2.,
          10.,   5.,   3.,   2.,   0.,   2.,   5.,   2.,   0.,   0.,   0.,   5.,
           0.,   0.,   0.,   0.,   1.,   2.,   0.,   0.,  36.,   1.,   5.,   0.,
           1.,  21.,   0.,   0.,   1.,   6.,   0.,   0.,   0.,   4.,  12.,   0.,
           5.,   2.,   0.,   1.,   1.,   0.,   0.,   0.,   0.,   5.,  11.,   7.,
           6.,   0.,  13.,   0.,   0.,   1.,   1.,   3.,  10.,   0.,   0.,   3.,
          22.,   7.,   0.,   1.,   5.,   1.,   8.,   2.,  16.,   0.,   1.,  20.,
          13.,   0.,   0.,   3.,   1.,   0.,   0.,   5.,   5.,   2.,   0.,   1.,
           0.,   2.,   2.,   2.,   0.,   6.,  12.,   0.,   0.,   0.,   5.,   0.,
           6.,   4.,   0.,   0.,   7.,   1.,   1.,   1.,   6.,  10.,   0.,   0.,
           0.,  19.,   4.,   0.,   0.,   4.,   0.,   0.,  50.,   1.,   1.,   0.,
           0.,

Seed set to 0


codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L tensor([[10.,  9.,  3.,  1.,  2.,  2.,  1.,  4., 11., 10.,  0.,  3.,  1.,  2.,
          4.,  2.,  3.,  2.,  3.,  2.,  3.,  4.,  1., 23.,  2.,  2., 13.,  8.,
          7.,  7.,  3.,  0., 14.,  6.,  6.,  3.,  2.,  2.,  1.,  4.,  7., 13.,
          3.,  2.,  2.,  3.,  0.,  8.,  4.,  5.,  2.,  2.,  7.,  2.,  5.,  4.,
          7.,  3., 10.,  4.,  6.,  3.,  3.,  2.,  1.,  4.,  3.,  1., 25.,  1.,
          2.,  2.,  3.,  2.,  3.,  5.,  5.,  1.,  3.,  2.,  2.,  5.,  3.,  4.,
          8.,  4.,  5.,  4.,  5.,  1.,  2.,  3.,  4.,  2.,  1.,  2.,  8.,  4.,
          4.,  3.,  2.,  0.,  6.,  2.,  2.,  3.,  7.,  1.,  1.,  2.,  1.,  1.,
          1.,  6.,  3.,  2.,  9., 17.,  2.,  3., 11.,  4.,  2., 10.,  4.,  1.,
          5.,  2.,  5.,  1., 16.,  4.,  1., 10.,  2., 10., 12.,  3.,  3.,  3.,
          1.,  3.,  2.,  5.,  1.,  5.,  8., 10., 10.,  6.,  1.,  2.,  5.,  2.,
          1.,  3., 11., 27., 16.,  3

Seed set to 0


codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L tensor([[ 6.,  7.,  4.,  6.,  4.,  6.,  3.,  4.,  8.,  7.,  5.,  4.,  3.,  3.,
          5.,  3.,  5.,  3.,  4.,  3.,  3.,  5.,  7., 12.,  3.,  2.,  7.,  6.,
          6.,  5.,  4.,  5.,  7.,  7.,  5.,  4.,  3.,  3.,  4.,  6.,  5., 10.,
          4.,  4.,  5.,  3.,  3.,  5.,  4.,  5.,  6.,  4.,  6.,  4.,  6.,  5.,
          7.,  7.,  7.,  4.,  4.,  3.,  5.,  2.,  1.,  5.,  3.,  5.,  8.,  2.,
          3.,  4.,  6.,  4.,  2.,  5.,  5.,  4.,  3.,  4.,  2.,  5.,  6.,  4.,
          5.,  6.,  7.,  5.,  5.,  4.,  2.,  3.,  4.,  4.,  4.,  5.,  5.,  4.,
          5.,  3.,  4.,  1.,  4.,  3.,  4.,  3.,  7.,  2.,  4.,  7.,  3.,  5.,
          6.,  4.,  4.,  7.,  7., 11.,  2.,  4.,  7.,  3.,  4.,  4.,  7.,  3.,
          6.,  5.,  5.,  3., 10.,  4.,  3.,  4.,  3.,  5.,  8.,  3.,  6.,  3.,
          3.,  6.,  3.,  6.,  1.,  4.,  7.,  6.,  8.,  5.,  4.,  2.,  4.,  4.,
          2.,  5.,  3.,  8.,  7.,  5

Seed set to 0


codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L tensor([[ 5.,  5.,  5.,  6.,  4.,  5.,  5.,  3.,  7.,  7.,  5.,  4.,  4.,  4.,
          6.,  4.,  6.,  5.,  4.,  3.,  4.,  5.,  7., 10.,  6.,  5.,  5.,  5.,
          5.,  4.,  5.,  5.,  6.,  6.,  5.,  5.,  3.,  4.,  3.,  7.,  5.,  8.,
          5.,  3.,  5.,  5.,  3.,  6.,  5.,  4.,  6.,  4.,  4.,  4.,  6.,  6.,
          5.,  5.,  7.,  5.,  6.,  4.,  4.,  3.,  3.,  5.,  3.,  6.,  7.,  3.,
          4.,  4.,  6.,  4.,  4.,  5.,  5.,  4.,  4.,  4.,  2.,  5.,  6.,  5.,
          5.,  5.,  7.,  6.,  5.,  3.,  2.,  3.,  4.,  5.,  2.,  5.,  5.,  4.,
          5.,  3.,  4.,  3.,  5.,  4.,  4.,  3.,  5.,  3.,  5.,  6.,  4.,  5.,
          5.,  3.,  4.,  6.,  6.,  5.,  3.,  5.,  4.,  3.,  4.,  5.,  6.,  4.,
          5.,  4.,  4.,  4.,  9.,  5.,  4.,  4.,  4.,  4.,  7.,  4.,  6.,  4.,
          3.,  6.,  3.,  5.,  2.,  4.,  6.,  6.,  6.,  6.,  3.,  4.,  3.,  5.,
          5.,  5.,  2.,  5.,  6.,  5

In [3]:
# Seed set to 0
# codevector_probs.shape torch.Size([2998, 320])
# codevector_probs.sum(0) tensor([10., 10.,  7.,  8., 11.,  9.,  9., 11., 10., 11., 12., 10.,  9.,  8.,
#          8.,  5.,  8.,  9., 10., 11., 10.,  9.,  9.,  7.,  8.,  9.,  7.,  8.,
#         10., 10.,  9., 10., 10.,  8.,  8.,  8.,  9., 10., 11., 11., 11.,  8.,
#         11.,  9.,  9., 10., 10.,  9., 10.,  7., 10.,  9.,  9., 11., 10.,  8.,
#         11.,  9., 10.,  9., 12., 11.,  9.,  8.,  9., 10.,  7.,  8.,  8., 10.,
#          9.,  9., 11., 10., 10.,  8., 11.,  8.,  6.,  9., 11.,  9., 10., 11.,
#         11.,  8.,  8., 12.,  9., 11.,  9., 12., 11., 12.,  9.,  8., 10., 10.,
#          9.,  8.,  8.,  7., 10.,  8.,  9.,  8.,  9.,  8.,  8., 13., 11.,  8.,
#          8.,  9.,  9.,  8.,  9.,  9., 11., 10., 11.,  8., 11.,  9., 10., 10.,
#          8., 10.,  6., 10., 10.,  6.,  9., 10.,  9.,  9.,  9.,  8., 13.,  9.,
#         10.,  9., 12., 10.,  7.,  9., 10.,  8.,  8., 11., 10.,  9.,  9.,  8.,
#         10., 10.,  7.,  6.,  9.,  5.,  8., 10., 10.,  8., 11.,  9., 10., 11.,
#          7.,  8.,  9.,  9.,  6.,  8., 10., 10.,  9., 12.,  9.,  9., 10.,  7.,
#          8., 10.,  8.,  9., 10.,  9., 12., 10., 12., 11.,  9., 10.,  9.,  9.,
#         10., 12., 10., 10.,  9.,  9., 14., 10.,  9.,  8., 11., 10., 11.,  9.,
#          9.,  8.,  9., 11., 11.,  8.,  9., 10.,  7., 10., 11., 10., 12., 11.,
#         10.,  8.,  9., 10., 11.,  7.,  8.,  7., 11.,  9., 11.,  9., 10.,  9.,
#          8.,  8.,  8., 12.,  9., 12., 12., 11., 10.,  8., 10., 10., 10.,  8.,
#          9.,  8., 12., 12., 12.,  9.,  9.,  5.,  8.,  9., 11.,  9., 10., 10.,
#          9., 11., 11.,  9.,  9.,  8., 10.,  9., 10.,  9.,  9.,  6.,  9.,  7.,
#         10.,  8., 10.,  8.,  9.,  9., 11., 10., 11., 10.,  9.,  8.,  9., 10.,
#         10., 10., 10.,  9.,  8., 14., 13.,  8., 10.,  9.,  8., 11.,  9.,  8.,
#         10.,  9.,  8., 10.,  6., 10., 12.,  9., 10.,  8.,  8., 11.])
# codevector_probs.sum(1) tensor([1., 1., 1.,  ..., 1., 1., 1.])
# tensor(168.0099)